In [1]:
import pandas as pd
import numpy as np
from textblob import TextBlob
import re
import csv
import sys
from nltk.stem.porter import PorterStemmer
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
from scipy.special import softmax
import urllib.request


In [2]:
data = pd.read_table("Task-1 tweets_1000.csv",header=None)


In [3]:
words=[]
for i in data[0]:
    words.append(i)

In [4]:
#Data Cleaning
use_stemmer =True
def preprocess_word(word):
    # Remove punctuation
    word = word.strip('\'"?!,.():;')
    # Convert more than 2 letter repetitions to 2 letter
    # funnnnny --> funny
    word = re.sub(r'(.)\1+', r'\1\1', word)
    # Remove - & '
    word = re.sub(r'(-|\')', '', word)
    return word


def is_valid_word(word):
    # Check if word begins with an alphabet
    return (re.search(r'^[a-zA-Z][a-z0-9A-Z\._]*$', word) is not None)


def handle_emojis(tweet):
    # Smile -- :), : ), :-), (:, ( :, (-:, :')
    tweet = re.sub(r'(:\s?\)|:-\)|\(\s?:|\(-:|:\'\))', ' EMO_POS ', tweet)
    # Laugh -- :D, : D, :-D, xD, x-D, XD, X-D
    tweet = re.sub(r'(:\s?D|:-D|x-?D|X-?D)', ' EMO_POS ', tweet)
    # Love -- <3, :*
    tweet = re.sub(r'(<3|:\*)', ' EMO_POS ', tweet)
    # Wink -- ;-), ;), ;-D, ;D, (;,  (-;
    tweet = re.sub(r'(;-?\)|;-?D|\(-?;)', ' EMO_POS ', tweet)
    # Sad -- :-(, : (, :(, ):, )-:
    tweet = re.sub(r'(:\s?\(|:-\(|\)\s?:|\)-:)', ' EMO_NEG ', tweet)
    # Cry -- :,(, :'(, :"(
    tweet = re.sub(r'(:,\(|:\'\(|:"\()', ' EMO_NEG ', tweet)
    return tweet


def preprocess_tweet(tweet):
    processed_tweet = []
    # Convert to lower case
    tweet = tweet.lower()
    # Replaces URLs with the word URL
    tweet = re.sub(r'((www\.[\S]+)|(https?://[\S]+))', ' URL ', tweet)
    # Replace @handle with the word USER_MENTION
    tweet = re.sub(r'@[\S]+', 'USER_MENTION', tweet)
    # Replaces #hashtag with hashtag
    tweet = re.sub(r'#(\S+)', r' \1 ', tweet)
    # Remove RT (retweet)
    tweet = re.sub(r'\brt\b', '', tweet)
    # Replace 2+ dots with space
    tweet = re.sub(r'\.{2,}', ' ', tweet)
    # Strip space, " and ' from tweet
    tweet = tweet.strip(' "\'')
    # Replace emojis with either EMO_POS or EMO_NEG
    tweet = handle_emojis(tweet)
    # Replace multiple spaces with a single space
    tweet = re.sub(r'\s+', ' ', tweet)
    words = tweet.split()

    for word in words:
        word = preprocess_word(word)
        if is_valid_word(word):
            word = str(porter_stemmer.stem(word))
            processed_tweet.append(word)
    
    return ' '.join(processed_tweet)

porter_stemmer = PorterStemmer()

In [5]:
processed_tweets=[]
for tweet in words:
    if preprocess_tweet(tweet) is None:
        x="NaN"
    else:
        x=preprocess_tweet(tweet)
    processed_tweets.append(x)

In [6]:
#Sentiment Analysis using Textbolb
Tweetdf = pd.DataFrame(columns=['Tweet','Sentiment'])

for tweet in processed_tweets:
    print(tweet)
    textanalysis = TextBlob(tweet)
    if textanalysis.sentiment[0]>0:
        status = "Positive"
        print ('Positive')
    elif textanalysis.sentiment[0]<0:
        print ('Negative')
        status = "Negative"
    else:
        print ('Neutral')
        status = "Neutral"
    Tweetdf = Tweetdf.append({'Tweet': tweet, 'Sentiment': status}, ignore_index=True)
Tweetdf.reset_index(drop= True,inplace= True)
Tweetdf.to_csv('/Users/jaide/Desktop/cleanedtweets.csv',index=None)

suck at social distanc
Positive


C:\Users\jaide\AppData\Local\Temp\ipykernel_8352\1125369384.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Tweetdf = Tweetdf.append({'Tweet': tweet, 'Sentiment': status}, ignore_index=True)
C:\Users\jaide\AppData\Local\Temp\ipykernel_8352\1125369384.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Tweetdf = Tweetdf.append({'Tweet': tweet, 'Sentiment': status}, ignore_index=True)
C:\Users\jaide\AppData\Local\Temp\ipykernel_8352\1125369384.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Tweetdf = Tweetdf.append({'Tweet': tweet, 'Sentiment': status}, ignore_index=True)
C:\Users\jaide\AppData\Local\Temp\ipykernel_8352\1125369384.py:15: FutureWarning: The frame.append method is deprecated and will be removed from 

jasmin strang share a messag of hope dure thi life of covid19 musicvideo product by moodolog if you like it dm me whi if you coronaviru stayathom oaklandfilmmak url
Neutral
i gotta fight these allergi in public to make sure peopl think i got corona
Positive
url on easter pleas rememb the poor and desol covid19 infrastructur lalov natgeo compass
Negative
user_ment i have a cute one made from recycl sari silk my friend got me in nepal
Positive
told my mom we should start to work from home due to corona no one els work in our offic but is but ya know work from home sound nice
Positive

Neutral
user_ment user_ment user_ment user_ment user_ment user_ment so use your logic of not pay ani attent to fact and detail trump call covid a and seven week later infect and kill american
Neutral
we are all in deep doo doo
Neutral
to be honest everyon wa scare of coronaviru and govt had to somehow convinc the public to compli inflat death number were use to keep us at home regardless the end result is w

C:\Users\jaide\AppData\Local\Temp\ipykernel_8352\1125369384.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Tweetdf = Tweetdf.append({'Tweet': tweet, 'Sentiment': status}, ignore_index=True)
C:\Users\jaide\AppData\Local\Temp\ipykernel_8352\1125369384.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Tweetdf = Tweetdf.append({'Tweet': tweet, 'Sentiment': status}, ignore_index=True)
C:\Users\jaide\AppData\Local\Temp\ipykernel_8352\1125369384.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Tweetdf = Tweetdf.append({'Tweet': tweet, 'Sentiment': status}, ignore_index=True)
C:\Users\jaide\AppData\Local\Temp\ipykernel_8352\1125369384.py:15: FutureWarning: The frame.append method is deprecated and will be removed from 

new rule if you attempt to make an appoint for a covid19 swab test you do not get to complain about test if made an attempt and been declin you can complain all you want
Positive
user_ment url
Neutral
there wa a concert effort by the media in coronaviru brief to harrass and attack user_ment user_ment wa there and is like a cancer in the room
Neutral
from passeng pleas do the right thing user_ment and extend travel fund expir date for the million of dollar in unus fund that are expir in the next few week and summer month coronavirustravel southwestairlin
Positive
corona viru stat as user_ment taught me in well the corona viru stat of url
Neutral
q to deal with a pandem can you make ani good decis a no becaus everi decis is a dilemma there is a ton of meander outrag about bad choic rather than a focu on reduc the chanc of the next pandem tail covid19 win head we lose
Positive
user_ment user_ment damn republican user_ment realli are respons
Neutral
thi thread
Neutral
url
Neutral
user_ment

...

enjoy the socialdistanc blue coronaviru url
Positive
continu fascin genom clade comparison in washington state show that case are part of the url
Neutral
user_ment they got from the care act but they are lose per month from sale tax fare and extra clean and expens relat to covid19 they are make tradeoff right now that i believ will be good for everyon in the longterm
Positive
home dad so i see you avoid the freshman but look like you avoid covid19 me well imma head back to la and eat my life away
Neutral
along with clean air water relationship
Positive
i just found out my sister is come home been on lockdown in the philippin for the last week the url
Neutral
user_ment user_ment user_ment user_ment user_ment user_ment user_ment user_ment url
Neutral
covid19 creat some other user_ment user_ment user_ment user_ment user_ment url doori gullyboy covid19 coronaviru lockdown stayathom rajivnemaindori shortcutproduct user_ment
Negative
user_ment pleas revis your polici regard nurs wear n95 whe

C:\Users\jaide\AppData\Local\Temp\ipykernel_8352\1125369384.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Tweetdf = Tweetdf.append({'Tweet': tweet, 'Sentiment': status}, ignore_index=True)
C:\Users\jaide\AppData\Local\Temp\ipykernel_8352\1125369384.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Tweetdf = Tweetdf.append({'Tweet': tweet, 'Sentiment': status}, ignore_index=True)
C:\Users\jaide\AppData\Local\Temp\ipykernel_8352\1125369384.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Tweetdf = Tweetdf.append({'Tweet': tweet, 'Sentiment': status}, ignore_index=True)
C:\Users\jaide\AppData\Local\Temp\ipykernel_8352\1125369384.py:15: FutureWarning: The frame.append method is deprecated and will be removed from 


Negative
so tell me releas peopl from prison becaus of covid but i go to the beach or see my friend fuck outta here
Negative
unfortun thi is about beer so we cant realli talk about it with our student but it a great articl and the idea are worth discuss url
Positive
who say no evid show that have coronaviru prevent a second infect url
Neutral
so sorri
Neutral
jesu
Neutral
welp stay safe folk bug out to the wilder soon get ahead of thi potenti crisi come thi far to let thi wipe me out in the popul that die from thi ckd compromisedimmun coronaviru staysaf
Positive
if onli tech influenc were thi self awar
Neutral
german footbal impend return thrown into chao as entir dynamo dresden squad are forc into day of quarantin after player test posit for coronaviru befor bundesliga side were due to play hannov it may throw the bundesliga return into jeopardi url
Negative


C:\Users\jaide\AppData\Local\Temp\ipykernel_8352\1125369384.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Tweetdf = Tweetdf.append({'Tweet': tweet, 'Sentiment': status}, ignore_index=True)
C:\Users\jaide\AppData\Local\Temp\ipykernel_8352\1125369384.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Tweetdf = Tweetdf.append({'Tweet': tweet, 'Sentiment': status}, ignore_index=True)
C:\Users\jaide\AppData\Local\Temp\ipykernel_8352\1125369384.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Tweetdf = Tweetdf.append({'Tweet': tweet, 'Sentiment': status}, ignore_index=True)
C:\Users\jaide\AppData\Local\Temp\ipykernel_8352\1125369384.py:15: FutureWarning: The frame.append method is deprecated and will be removed from 

user_ment order of magnitud greater danger of fatal if you are url
Positive
i will spend the next year in a spacesuit and a face mask on if i have to take away my juic
Neutral
i think ppl are scare of die from corona no one want flu like symptom for the next year of their life lol
Positive
jare kushner and stephen miller made lastminut chang to trump errorriddl coronaviru sp url via user_ment
Neutral
i will wuhan coronaviru kill more peopl in one day as death toll top sar cnn url
Positive
